In [ ]:
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

In [ ]:
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexRegion, HexWrapper, HexPosition, PrimitiveDemo, hexBackground
from HexMagic.styles import StyleCSS

In [ ]:
from HexMagic.hydrology import DrainageBasins, Watershed
from HexMagic.climate import Climate, TerrainFactory
from HexMagic.terrain import Terrain
from HexMagic.terrainpatterns import TerrainPatterns

In [ ]:
def showCarve(sampleMap:Terrain,rivers,showFlow=False):

    #print(f"working with {len(rivers)} number of rivers")

    fills=["#1100ffe3","#1100ff91","#2a25727e"]
    riverStyle = StyleCSS("river",fill="none",stroke="blue",stroke_width=4)
    
    
    sampleMap.hexGrid.adjustRadius(15)
    aRender = sampleMap.hexGrid.builder
    rivSVG = ""
 
    colors = []
    legends = []
    for i, c  in enumerate(fills):
        aStyle = StyleCSS(f"river{i}",fill="none",stroke=c ,stroke_width=4)
        colors.append(aStyle)
        aRender.add_style(aStyle)

        aStyle = StyleCSS(f"Level_{i+1}",fill=c,stroke="#000000" ,stroke_width=2)
        legends.append(aStyle)
        aRender.add_style(aStyle)
 
    for i, stream in enumerate(rivers):
        #print(f"on river {i}")
        rivSVG += stream.svg(styles = colors)

    aRender.layers = []
    sampleMap.colorMap()

    sgrid = sampleMap.hexGrid
    #sgrid.builder.adjust("regions", sgrid.styleLayerOrdered(
    #    styles=sampleMap.colorLevels,
    #    f=unique_windy_edge(iterations=3)))

    sampleMap.add_climate_overlay(scale=0.08)
    sampleMap.add_rain_overlay()

    #sampleMap.hexGrid.update()
    if showFlow:
        sampleMap.addCoast()
        aRender.adjust("flows",sampleMap.flow_diagram())
    #else:
    #    aRender.adjust("contors", sampleMap.contorOverlay(commonStroke="black"))
    aRender.adjust("rivers", rivSVG)
    aRender.adjust("legend",drawLegend(legends))
    
    #return rivSVG
    return  sampleMap.hexGrid.builder.show()

###

In [ ]:
world = TerrainFactory.create_ocean_world(
    bounds= MapRect(MapCord(0, 0), MapSize(700, 400)),
    preset='temperate',
    radius=15,
    lon_span=10.0,
    num_plates=18,
    subdivisions=3,
    ocean_fraction=0.3,
    oceanic_sides=['W','N','E'],  # Ocean on east and west
    terrain_age='old',  # Sharp, dramatic features
    formation_type='rolling',  # Creates ridge formations
    elevation_scale=6.5,  # Exaggerate the heights
    erosion_age=0.1,  # Minimal erosion for sharp peaks
    num_lakes=1,
    seed=23
)

# Compute climate
world.terrain.recomputeClimate()

Done at iter 1: 1 lakes


In [ ]:
world.terrain.colorMap()
world.terrain.hexGrid.update()
world.terrain.hexGrid.builder.show()

In [ ]:
terrain = world.terrain
grid = terrain.hexGrid
canvas = grid.builder

canvas.adjust("flows",terrain.flow_diagram())
canvas.show()

In [ ]:
basin = DrainageBasins(terrain)
rivers = basin.get_major(8)

river_svg = ""
for river in rivers:
    river_svg += river.simplify(3).draw()
canvas.adjust("rivers",river_svg)
canvas.show()

In [ ]:
terrain.add_climate_overlay(scale=0.3,layer_name="flows")
canvas.show()